Summary : using a surrogate to show how the homeostasis recovers from a deviation to equiprobability. Little nb_quant, slower decrease of coeffs. Homeostasis should work better.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
np.set_printoptions(precision=6, suppress=True, threshold=np.inf)
%load_ext autoreload
%autoreload 2

### 1. Learning a good dictionnary

In [ ]:
matname = '2017-05-30_surrogate_DEBUG'
DEBUG_DOWNSCALE, verbose = 10, 10
matname = '2017-05-30_surrogate'
DEBUG_DOWNSCALE, verbose = 1, 10

do_sym = False
C = 4
n_dictionary = 18**2
nb_quant = 512


N_image = 10000//DEBUG_DOWNSCALE
database = '../../2017-02-07_UnsupervisedLearning/database'
l0_sparseness = 10
N_boost=n_dictionary//2 
K_boost = 4.
rho_coeff = .9
do_plots = True
i_sample = 13

eta_homeo = .001*DEBUG_DOWNSCALE
n_step = 1000//DEBUG_DOWNSCALE

In [ ]:
from shl_scripts.shl_experiments import SHL
list_figures  = ['show_dico']#, 'plot_variance',  'plot_variance_histogram',  'time_plot_prob',  'time_plot_kurt',  'time_plot_var']
shl = SHL(n_dictionary=n_dictionary, database=database, DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, verbose=verbose, C=C, do_sym=do_sym, nb_quant=nb_quant)
data = shl.get_data(matname=matname)
dico = shl.learn_dico(matname=matname, list_figures=list_figures)

No cache found /tmp/data_cache/2017-05-30_surrogate_data: Extracting data... Extracting data..Mdn_land53.jpg, Fda_urb540.jpg, Bda_art1079.jpg, Fda_A223013.jpg, Fdn_land317.jpg, Mdn_land750.jpg, Hdn_text6.jpg, Mdn_natu169.jpg, Mdn_N228077.jpg, Hda_obj96.jpg, Fdn_natu815.jpg, Bdn_natu5.jpg, Bdn_land861.jpg, Hda_obj416.jpg, Mdn_nat367.jpg, Bdn_text1.jpg, Mdn_N347004.jpg, Hdn_objn217.jpg, Fdn_land951.jpg, Bda_art1271.jpg, Fdn_natu176.jpg, Mdn_for72.jpg, Hda_obj414.jpg, Mda_urb324.jpg, Hda_obj296.jpg, Bda_int211.jpg, Mda_enc30.jpg, Hda_obj293.jpg, Hdn_N115088.jpg, Bdn_land376.jpg, Bdn_text5.jpg, Hdn_objn180.jpg, Fdn_open13.jpg, Hdn_N124065.jpg, Fda_art1180.jpg, Fda_art1464.jpg, Bda_art145.jpg, Fda_A463059.jpg, Fdn_nat895.jpg, Bda_art621.jpg, Hdn_objn200.jpg, Hda_obj172.jpg, Fdn_open17.jpg, Bdn_text7.jpg, Fda_A517086.jpg, Mdn_text113.jpg, Mdn_N228075.jpg, Mdn_land159.jpg, Fda_art772.jpg, Fda_A683068.jpg, Hda_obj71.jpg, Fda_art1418.jpg, Bda_art1117.jpg, Bda_room28.jpg, Bdn_for82.jpg, Fda_city

### 1.0 Analysis of present coding

### 2. Generate an image from the dictionary

#### 2.1. Generate a random sparse vector

In [ ]:
from shl_scripts.shl_tools import generate_sparse_vector
sparse_vector, rho_vector = generate_sparse_vector(N_image, l0_sparseness, dico.n_dictionary, N_boost=N_boost, K_boost=K_boost, rho_coeff=rho_coeff)
print ('Sparse vector = \n', sparse_vector[i_sample, : ], ' and the respetive rho of each coeff \n', rho_vector[i_sample, : ])
print('List of non-zero coeffficients given by their addresses', np.nonzero(sparse_vector[i_sample, : ]))

In [ ]:
print ('Rho of each non-zero coeffficients given by their addresses', rho_vector[i_sample, np.nonzero(sparse_vector[i_sample, : ]) ] )

#### 2.2. Generate an Image as a linear combination of the sparse vector and the dictionary

In [ ]:
def generate_images(dico, vector):
    image = dico.dictionary.T @ vector.T
    return image.T

image = generate_images(dico, sparse_vector)
print(image.shape)

#### 2.3. Plot the image

In [ ]:
n_pixels = dico.dictionary.shape[1]
n_pixels_sqrt = int(np.sqrt(n_pixels))
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(image[i_sample, :].reshape(n_pixels_sqrt, n_pixels_sqrt), cmap='gray', interpolation="nearest")
plt.show()

### 3. Matching Pursuit Coding of the synthetic image

In [ ]:
from shl_scripts.shl_encode import mp 
mp_sparse_code = mp(image, dico.dictionary, l0_sparseness, verbose=True, C=C, do_sym=do_sym)
mp_sparse_code.shape

In [ ]:
print('Sparse vector = ', mp_sparse_code[i_sample, : ])
print('List of non-zero coeffficients given by their addresses', np.nonzero(sparse_vector[i_sample, : ]))
print('List of non-zero coeffficients given by their addresses', np.nonzero(mp_sparse_code[i_sample, :]))

In [ ]:
from shl_scripts.shl_tools import plot_scatter_MpVsTrue

plot_scatter_MpVsTrue(sparse_vector, mp_sparse_code);

In [ ]:
from shl_scripts.shl_learn import get_P_cum
from shl_scripts.shl_tools import plot_P_cum
fig, ax = plot_P_cum(get_P_cum(mp_sparse_code, C=C, do_sym=do_sym))
ax.set_ylim(0.9, 1.01);

In [ ]:
sparse_vector_, rho_vector_ = generate_sparse_vector(N_image, l0_sparseness, dico.n_dictionary, rho_coeff=.7, N_boost=0, K_boost=1., seed=420, do_sym=do_sym)
image_ = generate_images(dico, sparse_vector_)
mp_sparse_code_ = mp(image_, dico.dictionary, l0_sparseness, verbose=True, C=C, do_sym=do_sym)
plot_scatter_MpVsTrue(sparse_vector_, mp_sparse_code_);

In [ ]:
fig, ax = plot_P_cum(get_P_cum(mp_sparse_code_, C=C, do_sym=do_sym))
ax.set_ylim(0.9, 1.01);

In [ ]:
print ('Ranks of each non-zero coeffficients given by their addresses', rho_vector[i_sample, np.nonzero(sparse_vector[i_sample, : ]) ] )
print ('Ranks of each non-zero coeffficients given by their addresses', rho_vector[i_sample, np.nonzero(mp_sparse_code[i_sample, : ]) ] )
from scipy.stats import spearmanr
print ('Spearman coeficient ', spearmanr(sparse_vector[:2, : ], mp_sparse_code[:2, : ], axis=1) )
spearR = [spearmanr(sparse_vector[i_test, : ], mp_sparse_code[i_test, : ])[0] for i_test in range(N_image) ]
print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )

In [ ]:
spearR = [spearmanr(rho_vector[i_test, : ], mp_sparse_code[i_test, : ])[0] for i_test in range(N_image) ]
print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )

In [ ]:
spearR = [spearmanr(rho_vector[i_test, np.nonzero(sparse_vector[i_test, : ])], 
                    mp_sparse_code[i_test, np.nonzero(sparse_vector[i_test, : ]) ])[0] for i_test in range(N_image) ]
print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )

In [ ]:
from shl_scripts.shl_tools import plot_proba_histogram
from shl_scripts.shl_encode import z_score, prior
C = 5
n_samples, nb_filter = mp_sparse_code.shape
P_cum = np.linspace(0, 1, nb_quant, endpoint=True)[np.newaxis, :] * np.ones((nb_filter, 1))
stick = np.arange(dico.n_dictionary)*nb_quant

In [ ]:
rho_sparse_code = z_score(P_cum, prior(mp_sparse_code, C=C), stick)
print('Sparse vector = ', mp_sparse_code[i_sample, : ])
print('rho_true vector = ', rho_vector[i_sample, : ])
print('rho_hat  vector = ', rho_sparse_code[i_sample, : ])
spearR = [spearmanr(rho_vector[i_test, : ], rho_sparse_code[i_test, : ])[0] for i_test in range(N_image) ]
print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )

In [ ]:
help(spearmanr)

In [ ]:
rho_sparse_code[rho_sparse_code==0] = np.nan
spearR = [spearmanr(rho_vector[i_test, : ], rho_sparse_code[i_test, : ], nan_policy='omit')[0] for i_test in range(N_image) ]
print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )

In [ ]:
def evaluate(P_cum, my_sparse_code):
    print('mean Errors per image = ', np.sum(1 - ((np.abs(my_sparse_code)>0)*1) == sparse_vector) / N_image )
    fig, ax = plot_proba_histogram(my_sparse_code)
    rho_sparse_code = z_score(P_cum, prior(my_sparse_code, C=C), stick)
    spearR = [spearmanr(rho_vector[i_test, : ], my_sparse_code[i_test, : ])[0] for i_test in range(N_image) ]
    print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )
    rho_sparse_code[rho_sparse_code==0] = np.nan
    spearR = [spearmanr(rho_vector[i_test, : ], rho_sparse_code[i_test, : ], nan_policy='omit')[0] for i_test in range(N_image) ]
    print ('Spearman coeficient ', np.mean(spearR), '+/-', np.std(spearR) )
    return np.mean(spearR), np.std(spearR)
    
evaluate(P_cum, mp_sparse_code)    

### 4. Pcum Coding of the synthetic image

In [ ]:
from shl_scripts.shl_learn import update_P_cum

result = []
for i in range(n_step):
    Pcum_sparse_code = mp(image, dico.dictionary, l0_sparseness, P_cum=P_cum, C=C, do_sym=do_sym)
    P_cum = update_P_cum(P_cum, code=Pcum_sparse_code, eta_homeo=eta_homeo, nb_quant=nb_quant, C=C, do_sym=do_sym, verbose=False)
    if i % (n_step//20) == 0:
        print('Learning step', i)
        result.append([i, evaluate(P_cum, Pcum_sparse_code)])


In [ ]:
#P_cum = np.linspace(0, 1, nb_quant, endpoint=True)[np.newaxis, :] * np.ones((nb_filter, 1))
fig, ax = plot_P_cum(P_cum, verbose=False);
ax.set_ylim(0.9, 1.01);

### conclusion: plots

In [ ]:
if do_plots:
    figsize = (8, 3)
    from shl_scripts.shl_tools import plot_P_cum
    #P_cum = np.linspace(0, 1, nb_quant, endpoint=True)[np.newaxis, :] * np.ones((nb_filter, 1))
    fig, ax = plot_P_cum(P_cum, verbose=False);
    fig.set_size_inches(figsize)
    plt.tight_layout()
    ax.set_ylim(0.85, 1.01);
    fig.savefig('z_score.pdf')


    fig, ax = plot_proba_histogram(mp_sparse_code)
    fig.set_size_inches(figsize)
    plt.tight_layout()
    ax.set_xlim(0, shl.n_dictionary)
    fig.savefig('PDF_nohomeo.pdf')

    fig, ax = plot_proba_histogram(Pcum_sparse_code)
    fig.set_size_inches(figsize)
    plt.tight_layout()
    ax.set_xlim(0, shl.n_dictionary)
    fig.savefig('PDF_homeo.pdf')
